In [1]:
from dotenv import find_dotenv, load_dotenv
import os
load_dotenv(find_dotenv())

groq_api_key = os.environ['GROQ_API_KEY']

In [2]:
from langchain_groq import ChatGroq
llama3 = ChatGroq(api_key = groq_api_key, model = "llama-3.1-70b-versatile")
llama3

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017F7F0DCBC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017F7F166D50>, model_name='llama-3.1-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
llama3.invoke("Halo llama! Sebutkan 4 contoh variabel makro ekonomi!")

AIMessage(content='Halo! Berikut adalah 4 contoh variabel makro ekonomi:\n\n1. **Pertumbuhan Ekonomi (PDB)**: Pertumbuhan ekonomi diukur dengan menggunakan Produk Domestik Bruto (PDB) yang menunjukkan total nilai barang dan jasa yang dihasilkan dalam suatu negara dalam satu tahun.\n\n2. **Inflasi**: Inflasi adalah kenaikan harga barang dan jasa secara umum dalam jangka waktu tertentu. Inflasi diukur dengan menggunakan Indeks Harga Konsumen (IHK).\n\n3. **Tingkat Pengangguran**: Tingkat pengangguran adalah persentase penduduk yang tidak bekerja dan mencari pekerjaan. Tingkat pengangguran dihitung dengan menggunakan data jumlah penduduk yang tidak bekerja dan jumlah penduduk yang berusia kerja.\n\n4. **Neraca Perdagangan**: Neraca perdagangan adalah perbedaan antara nilai ekspor dan impor suatu negara dalam jangka waktu tertentu. Neraca perdagangan yang surplus menunjukkan bahwa nilai ekspor lebih besar daripada nilai impor, sedangkan neraca perdagangan yang defisit menunjukkan bahwa nil

In [6]:
from langchain_core.tools import tool, StructuredTool
import yfinance as yf

@tool
def company_address(ticker: str) -> str:
    """
    Returns company address for input ticker.
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return " ".join([info[key] for key in ['address1','city','state','zip','country']])

@tool
def fulltime_employees(ticker: str) -> int:
    """
    Returns fulltime employees count for input ticker.
    e.g. company_address: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['fullTimeEmployees']

@tool
def last_close_price(ticker: str) -> float:
    """
    Returns last close price for input ticker.
    e.g. company_address: MSFT
    Returns last close price for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['previousClose']

@tool
def EBITDA(ticker: str) -> float:
    """
    Returns EBITDA for input ticker.
    e.g. company_address: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['ebitda']

@tool
def total_debt(ticker: str) -> float:
    """
    Returns total debt for input ticker.
    e.g. company_address: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['totalDebt']

@tool
def total_revenue(ticker: str) -> float:
    """
    Returns total revenue for input ticker.
    e.g. company_address: MSFT
    Returns total revenue for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['totalRevenue']

@tool
def debt_to_equity_ratio(ticker: str) -> float:
    """
    Returns debt to equity ratio for input ticker.
    e.g. company_address: AAPL
    Returns debt to equity ratio for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info['debtToEquity']

tools = [
 company_address,
 fulltime_employees,
 last_close_price,
 EBITDA,
 total_debt,
 total_revenue,
 debt_to_equity_ratio
]

In [7]:
llama3_with_tools = llama3.bind_tools(tools, tool_choice='auto')

In [8]:
ai_msg = llama3_with_tools.invoke('Hello, llama!')
print(ai_msg.content)
print(ai_msg.tool_calls)

Hello. How can I help you today?
[]


In [10]:
ai_msg = llama3_with_tools.invoke("What's the EBITDA of Microsoft?")
print(ai_msg.content)
print(ai_msg.tool_calls)
EBITDA.invoke(ai_msg.tool_calls[0]['args'])


[{'name': 'EBITDA', 'args': {'ticker': 'MSFT'}, 'id': 'call_62eg', 'type': 'tool_call'}]


129433001984